# aws_a2i Usage Example

In [1]:
import os
from pathlib import Path

from rich import print as rprint
from boto_session_manager import BotoSesManager

import aws_a2i.api as aws_a2i

First, we define some constant values

In [2]:
# the AWS boto session manager
# see https://pypi.org/project/boto-session-manager/ for more information
bsm = BotoSesManager(profile_name="awshsh_app_dev_us_east_1")

dir_here = Path(os.getcwd()).absolute()

# where you store the task template (the .liquid file)
path_task_template = dir_here.parent.joinpath("task-ui.liquid")
# where you store the rendered task ui HTML
path_task_ui_html = dir_here.parent.joinpath("task-ui.html")

path_task_template = dir_here.parent.joinpath("task-ui.liquid")

play = aws_a2i.Play(
    aws_account_id=bsm.aws_account_id,
    aws_region=bsm.aws_region,
    # task UI template name
    task_template_name="aws-a2i-example",
    task_template_content=path_task_template.read_text(),
    # Human Review Workflow name
    flow_definition_name="aws-a2i-example",
    # The IAM role for Human Review Workflow, you have to create it yourself
    flow_execution_role_arn=f"arn:aws:iam::{bsm.aws_account_id}:role/all-services-admin-role",
    # Where you store HIL output data
    output_bucket=f"{bsm.aws_account_id}-{bsm.aws_region}-data",
    output_key="projects/aws_a2i-project/hil-output/",
    # the Ground Truth private team ARN
    private_team_arn=f"arn:aws:sagemaker:{bsm.aws_region}:{bsm.aws_account_id}:workteam/private-crowd/my-private-team",
)

Preview Task UI Template, in this example, we build a reimbursement request review system.

Given a reimbursement request, approve or deny it.

```
{
        "date_of_expense": "2020-01-01",
        "payment_method": "Credit Card",
        "purpose_of_expenditure": "Business Meal",
        "amount": "36.99",
}
```

In [3]:
rprint(path_task_template.read_text())

<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
  <full-instructions header="Instructions">
    <p>Check the following data points: Date of expense, Payment method, Purpose of expenditure, amount. </p>
  </full-instructions>

  <short-instructions>
    <p>Review this reimbursement request, approve or deny it.</p>
  </short-instructions>

  <h2>Reimbursement request</h2>
  <ul>
    <li>Date of expense = {{ task.input.date_of_expense }}</li>
    <li>Payment method = {{ task.input.payment_method }}</li>
    <li>Purpose of expenditure = {{ task.input.purpose_of_expenditure }}</li>
    <li>Amount = $ {{ task.input.amount }}</li>
  </ul>

  <h2>Approve or deny it</h2>

  <input type="radio" id="approve" name="decision" value="approve" required>
  <label for="approve">Approve</label>

  <input type="radio" id="deny" name="decision" value="deny" required>
  <label for="deny">Deny</label>

  <br>

</crowd-form>

Deploy Task UI Template

In [4]:
play.deploy_hil_task_template(bsm=bsm, verbose=True);

🚀 Deploy Human in Loop task template
  preview at https://us-east-1.console.aws.amazon.com/sagemaker/groundtruth?region=us-east-1#/a2i/worker-task-templates/aws-a2i-example
  a HIL task template with the same content already exists, do nothing.


Then we can deploy the Human Review Workflow definition

In [5]:
play.deploy_flow_definition(
    bsm=bsm,
    task_description="Review reimbursement request",
    task_count=1,
);

🚀 Deploy Human review workflow definition, it may takes 30 sec ~ 1 minute
  preview at https://us-east-1.console.aws.amazon.com/sagemaker/groundtruth?region=us-east-1#/a2i/human-review-workflows/aws-a2i-example
  No configuration changed, do nothing.


Now the human review workflow is ready to use, let's start some human loop task and review some reimbursement request!

In [6]:
workspace_signin_url = play.get_workspace_signin_url(bsm=bsm)
print(f"Login to the workspace: {workspace_signin_url}")
input_data = {
    "date_of_expense": "2020-01-01",
    "payment_method": "Credit Card",
    "purpose_of_expenditure": "Business Meal",
    "amount": "36.99",
}
play.start_human_loop(bsm=bsm, input_data=input_data)

Login to the workspace: https://a1b2c3d4.labeling.us-east-1.sagemaker.aws
⏯ Start a Human Loop Task '1293ffcc-eca9-44ab-ae67-3c784f2ac8f5'
  You can preview HIL status at https://us-east-1.console.aws.amazon.com/sagemaker/groundtruth?region=us-east-1#/a2i/human-review-workflows/aws-a2i-example/human-loops/1293ffcc-eca9-44ab-ae67-3c784f2ac8f5


'arn:aws:sagemaker:us-east-1:111122223333:human-loop/1293ffcc-eca9-44ab-ae67-3c784f2ac8f5'

You can list human loops, and it returns user friendly ``HumanLoop`` objects.

In [7]:
result = play.list_human_loops(bsm=bsm)
for human_loop in result:
    if human_loop.is_completed():
        human_loop.get_details(bsm=bsm)
        rprint(human_loop)
        human_loop_output = human_loop.get_output(bsm=bsm)
        rprint(human_loop_output)
    else:
        rprint(human_loop)


HumanLoop(
    creation_time=datetime.datetime(2023, 7, 12, 0, 32, 56, 746000, tzinfo=tzutc()),
    failure_reason=None,
    failure_code=None,
    human_loop_status='InProgress',
    human_loop_name='1293ffcc-eca9-44ab-ae67-3c784f2ac8f5',
    human_loop_arn='arn:aws:sagemaker:us-east-1:111122223333:human-loop/1293ffcc-eca9-44ab-ae67-3c784f2ac8f5',
    flow_definition_arn='arn:aws:sagemaker:us-east-1:111122223333:flow-definition/aws-a2i-example',
    human_loop_output_s3uri=None,
    data={
        'HumanLoopName': '1293ffcc-eca9-44ab-ae67-3c784f2ac8f5',
        'HumanLoopStatus': 'InProgress',
        'CreationTime': datetime.datetime(2023, 7, 12, 0, 32, 56, 746000, tzinfo=tzutc()),
        'FlowDefinitionArn': 'arn:aws:sagemaker:us-east-1:111122223333:flow-definition/aws-a2i-example'
    }
)

HumanLoop(
    creation_time=datetime.datetime(2023, 7, 12, 0, 32, 6, 27000, tzinfo=tzutc()),
    failure_reason=None,
    failure_code=None,
    human_loop_status='InProgress',
    human_loop_name='75a77481-df0c-4476-98f2-d6773d0a2288',
    human_loop_arn='arn:aws:sagemaker:us-east-1:111122223333:human-loop/75a77481-df0c-4476-98f2-d6773d0a2288',
    flow_definition_arn='arn:aws:sagemaker:us-east-1:111122223333:flow-definition/aws-a2i-example',
    human_loop_output_s3uri=None,
    data={
        'HumanLoopName': '75a77481-df0c-4476-98f2-d6773d0a2288',
        'HumanLoopStatus': 'InProgress',
        'CreationTime': datetime.datetime(2023, 7, 12, 0, 32, 6, 27000, tzinfo=tzutc()),
        'FlowDefinitionArn': 'arn:aws:sagemaker:us-east-1:111122223333:flow-definition/aws-a2i-example'
    }
)

HumanLoop(
    creation_time=datetime.datetime(2023, 7, 12, 0, 30, 4, 543000, tzinfo=tzutc()),
    failure_reason=None,
    failure_code=None,
    human_loop_status='InProgress',
    human_loop_name='1dfbaa26-9e89-49be-aeaf-c36df05f5fd7',
    human_loop_arn='arn:aws:sagemaker:us-east-1:111122223333:human-loop/1dfbaa26-9e89-49be-aeaf-c36df05f5fd7',
    flow_definition_arn='arn:aws:sagemaker:us-east-1:111122223333:flow-definition/aws-a2i-example',
    human_loop_output_s3uri=None,
    data={
        'HumanLoopName': '1dfbaa26-9e89-49be-aeaf-c36df05f5fd7',
        'HumanLoopStatus': 'InProgress',
        'CreationTime': datetime.datetime(2023, 7, 12, 0, 30, 4, 543000, tzinfo=tzutc()),
        'FlowDefinitionArn': 'arn:aws:sagemaker:us-east-1:111122223333:flow-definition/aws-a2i-example'
    }
)

You can actually filter the human loops using their status (or any other attributes)

In [ ]:
for human_loop in aws_a2i.list_human_loops(
    bsm=bsm,
    flow_definition_arn=flow_definition_arn,
).filter(
    lambda human_loop: human_loop.human_loop_status == aws_a2i.HumanLoopStatusEnum.InProgress.value
):
    pass


You can use the ``aws_a2i.render_task_template`` function to preview the task UI in your browser without starting a real human in the loop and without deploying anything. It is purely Python and purely local.

![](./task-ui.png)

In [8]:
path_task_ui_html = dir_here.parent.joinpath("task-ui.html")
input_data = {
    "date_of_expense": "2020-01-01",
    "payment_method": "Credit Card",
    "purpose_of_expenditure": "Business Meal",
    "amount": "36.99",
}
aws_a2i.render_task_template(
    task_template_content=path_task_template.read_text(),
    input_data=input_data,
    path_task_ui_html=path_task_ui_html,
    preview=True,
)

ImportError: cannot import 'box' or 'liquid' package

The usage example is done. Let's clean up AWS resources to avoid cost.

In [3]:
play.remove_flow_definition(bsm=bsm)

🗑 Remove Human review workflow definition, it may takes 30 sec ~ 1 minute
  Preview at https://us-east-1.console.aws.amazon.com/sagemaker/groundtruth?region=us-east-1#/a2i/human-review-workflows/aws-a2i-example
start waiter, polling every 1 seconds, timeout in 30 seconds.
  on 5 th attempt, elapsed 5 seconds, remain 25 seconds ...
  ✅ Successfully delete flow definition 'aws-a2i-example'


In [4]:
play.remove_hil_task_template(bsm=bsm)

🗑 Remove Human in Loop task template
  verify at https://us-east-1.console.aws.amazon.com/sagemaker/groundtruth?region=us-east-1#/a2i/worker-task-templates/aws-a2i-example
  ✅ Successfully removed task ui template 'aws-a2i-example'


False